###### Downloading Packages

In [1]:
!pip install sacrebleu -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.0 MB/s eta 0:00:00


###### Calculating BLEU Score

In [9]:
import pandas as pd
from sacrebleu import corpus_bleu

# Read the CSV file
file_path = './Llama3_Results.csv'
df = pd.read_csv(file_path)

# Function to calculate BLEU score for a single row
def calculate_bleu_for_row(expert_translations, candidate_translation):
    references = [[exp] for exp in expert_translations]
    candidate = [candidate_translation]
    bleu_score = corpus_bleu(candidate, references).score
    return bleu_score

# Initialize variables to accumulate BLEU scores
total_bleu_score_default = 0
total_bleu_score_finalists = 0

# Prepare a new DataFrame to store results
results_df = pd.DataFrame(columns=['English', 'Igbo Default', 'Igbo Default Score', 'Igbo Finalists', 'Igbo Finalists Score'])

# Iterate through each row, calculate BLEU scores, and populate the new DataFrame
for index, row in df.iterrows():
    expert_translations = [row['Expert1 Translation'], row['Expert2 Translation'], row['Expert3 Translation']]

    # Calculate BLEU score for 'Igbo Default'
    bleu_score_default = calculate_bleu_for_row(expert_translations, row['Igbo Default'])
    total_bleu_score_default += bleu_score_default

    # Calculate BLEU score for 'Igbo Finalists'
    bleu_score_finalists = calculate_bleu_for_row(expert_translations, row['Igbo Finalists'])
    total_bleu_score_finalists += bleu_score_finalists

    # Create a temporary DataFrame for the current row and append it to results_df
    temp_df = pd.DataFrame({
        'English': [row['English']],
        'Igbo Default': [row['Igbo Default']],
        'Igbo Default Score': [bleu_score_default],
        'Igbo Finalists': [row['Igbo Finalists']],
        'Igbo Finalists Score': [bleu_score_finalists]
    })
    results_df = pd.concat([results_df, temp_df], ignore_index=True)

# Calculate averages and append as a new row in the same manner
average_bleu_score_default = total_bleu_score_default / len(df)
average_bleu_score_finalists = total_bleu_score_finalists / len(df)

average_row = pd.DataFrame({
    'English': ['Average Score'],
    'Igbo Default': [''],
    'Igbo Default Score': [average_bleu_score_default],
    'Igbo Finalists': [''],
    'Igbo Finalists Score': [average_bleu_score_finalists]
})

results_df = pd.concat([results_df, average_row], ignore_index=True)

# Display the final DataFrame
results_df

,English,Igbo Default,Igbo Default Score,Igbo Finalists,Igbo Finalists Score
0,I'm praying every day for the country.,Ahiajoku! N'anya ka anyị na-akpọrọ ọrụ ọma mak...,4.932352,Ana m ekpe ekpere kwa ụbọchị maka mba a.,66.063286
1,The Nigerian president Muhammadu Buhari has to...,Ahiajoku! Nne na-akpa Chibok nwanyi 5 afọ gara...,6.612237,Onyeisiala Muhammadu Buhari agwaala ndi nne na...,25.921705
2,Five Nigerians have been sued,Otu n'ime Nigeria abuo agha.,8.116698,Aka akpọọla ndị Naịjiria ise ikpe,9.652435
3,Zamfara primaries: We have those who will cont...,"Otu na-eme primaries na-eto na-eto, anyị na-em...",2.554050,Zamfara primaries: Anyị nwere ndị ga-azọ ọkwa ...,83.070185
4,Matawalle represented by the Secretary to the ...,"Onye isi Matawalle, site na Secretary to the s...",7.771324,Matawalle n' onye ọrụ gọọmentị steeti nọ n' ọk...,16.735917
5,"Priests are like helpers, they are not suppose...","Olu na ndị mmụọkarị, ha na-ahụ maka ọrụ, ha na...",1.979265,"Ndị ụka dị ka ndị enyemaka, ha ekwesighi inwe ...",44.500507
6,But Kenneth Umeru said that a lot of men do no...,Ahiajoku! (Ah-ee-ah-joh-koo) Nwoke a na-akpa n...,0.943228,Mana Kenneth Umeru kwuru na ọtụtụ nwoke anaghị...,40.902739
7,Neymar poses a threat to Real Madrird players ...,O nwere onye na-eme aka na Neymar na-akpa ọkwa...,9.562407,Neymar na-akpata ụdị ọgbaghara nye ndị Real Ma...,8.248473
8,A man cannot break colanut in the place he mar...,"Onye aghala ka ọkụlọ na ọrụ ya na ọdịnihu ya, ...",6.468491,Nwoke anaghị etinye ọjị n'ụlọ di ya maọbụ n'ọg...,6.041873
9,the association of anti child trafficking know...,"Ahiajoku! NAPTIP, agha mmuo, nke a na-akpọ NAP...",2.372094,Ndị NAPTIP kpụrụ otu onye dọkịta ọgwụ n' ike ọ...,1.862940


###### Saving to CSV

In [10]:
# Specify the filename for the new CSV
output_file_path = './results_bleu_scores.csv'

# Save results_df to a CSV file
results_df.to_csv(output_file_path, index=False)

print(f'Results saved to {output_file_path}')

Results saved to ./results_bleu_scores.csv
